c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
import sys
import mysql.connector
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import os
from dotenv import load_dotenv
load_dotenv()
#id del pedido en cuestion
id=str(sys.argv[1])
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')

a_color='#354F84'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
facturas=pd.read_sql("""Select factures.* ,
    customers.alias,customers.customer_suburb, customers.clave,
    internal_orders.invoice, internal_orders.payment_conditions,internal_orders.noha,
    internal_orders.category,internal_orders.description,internal_orders.status as estado,
    coins.exchange_sell, coins.code, coins.symbol,
    capturistas.name as capturista, revisores.iniciales as revisor, autorizadores.iniciales as autorizador
    from ((((((
    factures inner join internal_orders on internal_orders.id = factures.order_id) 
    inner join customers on customers.id = internal_orders.customer_id )
    inner join coins on internal_orders.coin_id = coins.id)
    left join cobros on factures.id=cobros.facture_id)
    left join users as capturistas on cobros.capturo=capturistas.id)
    left join users as revisores on cobros.reviso=revisores.id)
    left join users as autorizadores on cobros.autorizo=autorizadores.id
     """,cnx)

writer = pd.ExcelWriter('storage/report/consecutivo_factura1.xlsx', engine='xlsxwriter')
workbook = writer.book

##FORMATOS PARA EL TITULO------------------------------------------------------------------------------
rojo_l = workbook.add_format({
    'bold': 0,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    #'fg_color': 'yellow',
    'font_color': 'red',
    'font_size':16})
negro_s = workbook.add_format({
    'bold': 0,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':12})
negro_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':13,
    
    'text_wrap': True,
    'num_format': 'dd/mm/yyyy'}) 
rojo_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'red',
    'font_size':13})      

#FORMATOS PARA CABECERAS DE TABLA --------------------------------
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'center',
    'fg_color': 'yellow',
    'border': 1,})

blue_header_format = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'num_format': '[$$-409]#,##0.00',
    'border': 1})
blue_header_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'num_format': '[$$-409]#,##0.00',
    'border': 1,
    'font_size':13})
blue_footer_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'num_format': '[$$-409]#,##0.00',
    'font_size':11})
red_header_format = workbook.add_format({
    'bold': True,
    'bg_color': b_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1})

red_header_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': b_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'font_size':13})


#FORMATOS PARA TABLAS PER CE------------------------------------

blue_content = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})

blue_content_bold = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11,
    'border_color':a_color,
    'num_format': '[$$-409]#,##0.00'})

blue_content_bold_dll = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11,
    'bg_color': '#b4e3b1',
    'border_color':a_color,
    'num_format': '[$$-409]#,##0.00'})
blue_content_footer_dll = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'white',
    'font_size':11,
    'bg_color': '#356e31',
    'border_color':'white',
    'num_format': '[$$-409]#,##0.00'})
blue_content_footer = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'white',
    'font_size':11,
    'bg_color': '#3e5585',
    'border_color':'white',
    'num_format': '[$$-409]#,##0.00'})
blue_content_date = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':9,
    'border_color':a_color,
    'num_format': 'dd/mm/yyyy'})
#FOOTER FORMATS---------------------------------------------------------
observaciones_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color':'#BDD7EE',
    'border': 1})

blue_content_dll = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'bg_color': '#b4e3b1',
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})
total_cereza_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color':'#F4B084',
    'border': 1})

#dataframes

facturas['date'][0:1].to_excel(writer, sheet_name='Sheet1', startrow=7,startcol=2, header=False, index=False)
worksheet = writer.sheets['Sheet1']
#worksheet.set_column(2,19,15)
# Encabezado.

facturas['clave'] = facturas['clave'].replace({' ':''}, regex=True)
facturas['code'] = facturas['code'].replace({'MN':'MXN'}, regex=True)
worksheet.insert_image("E1", "img/logo/logo.png",{"x_scale": 0.5, "y_scale": 0.5})
worksheet.merge_range('G2:K2', 'TYRSA CONSORCIO S.A. DE C.V. ', rojo_l)
worksheet.merge_range('G3:K3', 'Soluciones en logistica interior', negro_s)
worksheet.merge_range('G4:K4', 'CONSECUTIVO DE FACTURAS' ,negro_b)
worksheet.merge_range('G5:K5', 'Control de Cobros por P.I.', rojo_b)

worksheet.merge_range('L2:M3', """FECHA DEL REPORTE             
DD/MM/AAAA""", negro_b)

import datetime
currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y")
worksheet.merge_range('N2:O3', date, negro_b)
#Dataframe yellow headers bitch xd
worksheet.merge_range('B6:B7', 'NOHA', blue_header_format)
worksheet.merge_range('C6:C7', """FECHA 
AAAA-MM-DD""", blue_header_format)
worksheet.merge_range('D6:D7', 'P.I. NO.', blue_header_format)
worksheet.merge_range('E6:E7', 'NUMERO DE PAGO', blue_header_format)
worksheet.merge_range('F6:F7', 'NUMERO TOTAL DE PAGOS', blue_header_format)
worksheet.merge_range('G6:G7', 'FACTURA FOLIO NO.', blue_header_format)
worksheet.merge_range('H6:H7', 'CLIENTE NO', blue_header_format)
worksheet.merge_range('I6:I7', 'NOMBRE CORTO CLIENTE', blue_header_format)
worksheet.merge_range('J6:J7', 'CATEGORIA EQUIPO', blue_header_format)
worksheet.merge_range('K6:K7', 'DESCRIPCION BREVE', blue_header_format)
worksheet.merge_range('L6:L7', 'UBI / SUC / TIENDA PROYECTO', blue_header_format)
worksheet.merge_range('M6:M7', 'TIPO DE MONEDA', blue_header_format)
worksheet.merge_range('N6:N7', 'TIPO DE CAMBIO', blue_header_format)

worksheet.merge_range('O6:P6', 'IMPORTE TOTAL SIN IVA', blue_header_format)
worksheet.write(6, 14, "M.N.(Equivalente)", blue_header_format)
worksheet.write(6, 15, "DLLS", blue_header_format)

worksheet.merge_range('Q6:Q7', 'CAPTURO', blue_header_format)
worksheet.merge_range('R6:R7', 'REVISO', blue_header_format)
worksheet.merge_range('S6:S7', 'AUTORIZO', blue_header_format)
worksheet.merge_range('T6:T7', 'STATUS', blue_header_format)

for i in range(0,len(facturas)):
    worksheet.write(7+i, 1,str(facturas['noha'].values[i]), blue_content)
    worksheet.write(7+i, 2,str(facturas['date'].values[i]), blue_content)
    worksheet.write(7+i, 3,str(facturas['invoice'].values[i]), blue_content)
    worksheet.write(7+i, 4,str(facturas['ordinal'].values[i]), blue_content)
    worksheet.write(7+i, 5,str(facturas['payment_conditions'].values[i]), blue_content)
    worksheet.write(7+i, 6,str(facturas['facture'].values[i]), blue_content)
    worksheet.write(7+i, 7,str(facturas['clave'].values[i]), blue_content)
    worksheet.write(7+i, 8,str(facturas['alias'].values[i]), blue_content)
    worksheet.write(7+i, 9,str(facturas['category'].fillna(' ').values[i].upper()), blue_content)
    worksheet.write(7+i, 10,str(facturas['description'].values[i].upper()), blue_content)
    worksheet.write(7+i, 11,str(facturas['customer_suburb'].values[i].upper()), blue_content)
    worksheet.write(7+i, 12,str(facturas['code'].values[i]), blue_content)
    worksheet.write(7+i, 13,str(facturas['exchange_sell'].values[i]), blue_content)
    if(facturas['exchange_sell'].values[i]>1):
        worksheet.write(7+i, 14, 0,blue_content)
        worksheet.write(7+i, 15, facturas['amount'].values[i]*facturas['exchange_sell'].values[i],blue_content_dll)
    else:
        worksheet.write(7+i, 14, facturas['amount'].values[i],blue_content)
        worksheet.write(7+i, 15,0,blue_content_dll)
    worksheet.write(7+i, 16, ' '.join(str(facturas['capturista'].values[i]).split(' ')[:2]),blue_content)
    worksheet.write(7+i, 17, str(facturas['revisor'].values[i]),blue_content)
    worksheet.write(7+i, 18, str(facturas['autorizador'].values[i]),blue_content)
    worksheet.write(7+i, 19, str(facturas['estado'].values[i].upper()),blue_content)
trow=8+len(facturas)

worksheet.merge_range('M'+str(trow)+':N'+str(trow), 'Total', blue_header_format_bold)

worksheet.write_formula('O'+str(trow),  '{=SUM(O8:O'+str(trow-1)+')}', blue_content_footer)
worksheet.write_formula('P'+str(trow),  '{=SUM(P8:P'+str(trow-1)+')}', blue_content_footer_dll)


worksheet.set_column('C:C',15)
worksheet.set_column('I:I',19)

worksheet.set_column('J:L',20)
worksheet.set_column('L:M',15)
worksheet.set_column('O:P',19)
worksheet.set_column('Q:T',20)

worksheet.set_landscape()
worksheet.set_paper(9)
worksheet.fit_to_pages(1, 0)  
workbook.close()



c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'upper'

In [27]:
facturas['estado'].unique()

array(['autorizado', 'CANCELADO'], dtype=object)

In [19]:
print(pd.read_sql(f'select factures.*,cobro_factures.cobro_id cobro_id from factures inner join cobro_factures on cobro_factures.facture_id=factures.id where order_id ={id}',cnx))

    id  order_id  ordinal facture    amount   status npagos  created_at  \
0  881       687        1  A 4176  323327.0  CAPTURA   None  2025-03-06   
1  881       687        1  A 4176  323327.0  CAPTURA   None  2025-03-06   
2  881       687        1  A 4176  323327.0  CAPTURA   None  2025-03-06   
3  881       687        1  A 4176  323327.0  CAPTURA   None  2025-03-06   

   updated_at        date  coin_id   tc  cobro_id  
0  2025-03-06  2025-02-28        1  1.0      1039  
1  2025-03-06  2025-02-28        1  1.0      1078  
2  2025-03-06  2025-02-28        1  1.0      1103  
3  2025-03-06  2025-02-28        1  1.0      1138  


c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:
400/28

14.285714285714286

In [10]:
2200/4

550.0

In [21]:
cobros=pd.read_sql("""Select cobros.* ,
    customers.customer,customers.customer_suburb, customers.clave,
    internal_orders.invoice, internal_orders.payment_conditions,
    internal_orders.category,internal_orders.description,internal_orders.status,
    coins.exchange_sell, coins.coin, coins.symbol,
    banks.bank_description,
    capturistas.name as capturista, revisores.name as revisor, autorizadores.name as autorizador
    from (((((((
    cobros inner join internal_orders on internal_orders.id = cobros.order_id) 
    inner join customers on customers.id = internal_orders.customer_id )
    inner join coins on internal_orders.coin_id = coins.id)
    left join users as capturistas on cobros.capturo=capturistas.id)
    left join users as revisores on cobros.reviso=revisores.id)
    left join users as autorizadores on cobros.autorizo=autorizadores.id))
    inner join banks on banks.id=cobros.bank_id """,cnx)

In [9]:
cobros.loc[cobros['order_id']==681]

NameError: name 'cobros' is not defined

In [7]:
459692.30/2

229846.15

In [20]:
orders.loc[orders['date'].astype(str)>str(date.strftime("%Y"))+'-01-01',"total"]

423    5.183228e+04
424    1.003168e+05
425    1.026716e+04
426    1.295720e+04
427    1.088080e+05
428    1.128912e+05
429    6.275600e+04
430    7.946000e+04
431    6.739600e+04
432    4.609944e+03
433    8.120000e+03
434    3.961400e+05
435    5.707200e+04
436    2.587960e+05
437    8.651280e+03
438    1.063720e+05
439    2.489360e+03
440    1.651724e+06
441    2.683672e+05
442    3.209720e+05
443    3.209720e+05
444    3.430120e+05
445    1.894280e+05
446    6.275832e+04
447    0.000000e+00
448    1.940100e+04
449    4.369720e+04
450    1.263240e+05
451    0.000000e+00
452    2.487040e+05
453    9.326400e+04
454    2.379160e+06
455    0.000000e+00
456    4.717720e+06
457    2.379160e+06
458    4.609944e+03
459    6.496000e+04
460    1.476680e+05
461    0.000000e+00
462    3.781600e+04
463    0.000000e+00
464    0.000000e+00
465    5.707200e+04
466    0.000000e+00
Name: total, dtype: float64

In [16]:
orders.loc[orders['date'].astype(str)>'2025-01-01','total'].sum()

14925724.8488

In [19]:
str(date.strftime("%Y"))+'01-01'

'202501-01'

In [ ]:
import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

year = datetime.date.today().year
quincena=12
month = np.ceil(quincena/ 2)
isFirstHalf = quincena % 2 != 0
startDate =  str(year)+"-"+str(int(month)).zfill(2)+"-01" if isFirstHalf else  str(year)+"-"+str(int(month)).zfill(2)+"-15"
endDate =  str(year)+"-"+str(int(month)).zfill(2)+"-14" if isFirstHalf else  str((datetime.datetime(year,int(month),1 )+relativedelta(months=1))-datetime.timedelta(days=1))[:10];

2025-06-15 2025-06-30


In [ ]:
#siguiente mes en dia primero


'2025-05-31'

In [11]:
'1'.zfill(2)

'01'

In [19]:
facturas.loc[facturas['id'].isin(cobros_facturas.loc[cobros_facturas['cobro_id'].isin(cobros['cobro_id'].values)].facture_id.values)]

,id,order_id,ordinal,facture,amount,status,npagos,created_at,updated_at,date,coin_id,tc
1,826,620,1,A 4115,818738.0,CAPTURA,None,2025-01-10,2025-01-10,2025-01-10,1,1.0
2,835,620,1,A 4125,1309980.0,CAPTURA,None,2025-01-27,2025-01-27,2025-01-21,1,1.0


In [20]:
facturas

,id,order_id,ordinal,facture,amount,status,npagos,created_at,updated_at,date,coin_id,tc
0,821,620,1,A 4110,818738.0,CAPTURA,None,2025-01-03,2025-01-03,2024-12-30,1,1.0
1,826,620,1,A 4115,818738.0,CAPTURA,None,2025-01-10,2025-01-10,2025-01-10,1,1.0
2,835,620,1,A 4125,1309980.0,CAPTURA,None,2025-01-27,2025-01-27,2025-01-21,1,1.0


In [23]:
818738.0+1309980.0

2128718.0

In [42]:
for i in cobros.cobro_id.unique():
    print(i)
    fac_ids=cobros_facturas.loc[cobros_facturas['cobro_id']==i].facture_id.unique()
    for j in fac_ids:
        print('',j)
        print(len(facturas.loc[facturas['id']==j,['id','facture']]))

214
 150
1
 159
1
215
 151
1
216
 152
1
 153
1
217
 154
1
 155
1
249
 157
1
 158
1
 212
1
483
 417
1
 426
0
 440
1
 441
1
 442
0
487
 374
1
 443
1
542
 455
1
573
 515
1
635
 515
1
665
 544
1


In [10]:
cobros.facture_id

AttributeError: 'DataFrame' object has no attribute 'facture_id'

In [32]:
all_facturas

NameError: name 'all_facturas' is not defined

In [18]:
cobros_facturas.loc[cobros_facturas['cobro_id']==679]

,cobro_id,facture_id,id,created_at,updated_at
639,679,167,1034,2025-01-26,2025-01-26


In [26]:
allcobros['ordinal']=range(1,len(allcobros)+1)

C:\Users\hp\AppData\Local\Temp\ipykernel_14000\141872390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allcobros['ordinal']=range(1,len(allcobros)+1)


In [27]:
allcobros

,cobro_id,order_id,id,created_at,updated_at,amount,comp,date,bank_id,customer_id,invoice,noha,seller_id,comision,total,ordinal
33,58,155,40,None,None,1252626.0,1862,2022-12-29,2,446,2950,2022,14,0.003,2505252.0,1
34,59,155,41,None,None,1002100.8,1940,2023-03-28,2,446,2950,2022,14,0.003,2505252.0,2
540,595,155,580,2024-03-01,2024-03-01,250525.2,2297,2024-03-01,2,446,2950,2022,14,0.003,2505252.0,3


In [ ]:
cobros[['invoive']]

,cobro_id,order_id,id,created_at,updated_at,amount,comp,date,bank_id,customer_id,invoice,noha,seller_id,comision,total
0,21,116,4,None,None,150000.00,1066,2020-02-24,2,506,2175,12020,2,0.030,356120.00
1,22,116,5,None,None,20000.00,1472,2021-08-16,2,506,2175,12020,2,0.030,356120.00
2,23,116,6,None,None,20000.00,1492,2021-09-17,2,506,2175,12020,2,0.030,356120.00
3,24,116,7,None,None,23200.00,1526,2021-11-12,2,506,2175,12020,2,0.030,356120.00
4,25,116,8,None,None,23200.00,1552,2021-12-23,2,506,2175,12020,2,0.030,356120.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,695,332,688,2024-05-17,2024-05-17,55482.68,2381,2024-05-16,1,418,3102,1412023,14,0.003,1090632.00
633,695,451,689,2024-05-17,2024-05-17,92624.84,2381,2024-05-16,1,418,3102,1412023,14,0.003,1090632.00
634,696,438,690,2024-05-17,2024-05-17,444048.00,2382,2024-05-16,2,337,3191,42,5,0.003,1110120.00
635,697,398,691,2024-05-17,2024-05-17,502194.16,2383,2024-05-17,2,424,3160,11,2,0.003,1062388.32
